In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import torch

In [ ]:
BASE_MODEL = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
ADAPTER_PATH = "./lora_adapters/empathy"  # 훈련된 adapter 경로

# 모델 및 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float16
)

# LoRA adapter 적용
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

In [ ]:
def run_empathy_inference(user_input):
    prompt = (
        "Instruct: 아래 내용을 감정적으로 공감하며 응답하세요.\n"
        f"Input: {user_input}\n"
        "Output:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# 예시 테스트
test_input = "요즘 너무 지치고 외롭습니다. 아무에게도 말할 수 없어요."
print(run_empathy_inference(test_input))